In [1]:
!pip install llama-index llama-index-llms-groq llama-index-embeddings-huggingface chromadb llama-index-vector-stores-chroma

In [4]:
import zipfile
import os

# Extract the zip file
zip_path = '/content/doc_dir.zip'
extract_path = '/content/doc_dir'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✓ Files extracted successfully!")

# List all extracted files
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith('.pdf'):
            print(f"Found PDF: {os.path.join(root, file)}")

✓ Files extracted successfully!
Found PDF: /content/doc_dir/doc_dir/Attention.pdf
Found PDF: /content/doc_dir/doc_dir/BERT.pdf
Found PDF: /content/doc_dir/__MACOSX/doc_dir/._Attention.pdf
Found PDF: /content/doc_dir/__MACOSX/doc_dir/._BERT.pdf


In [5]:
#importing the dependencies

from llama_index.core import Settings
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader, ServiceContext, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb

In [6]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("punkt_tab")
nltk.download("stopwords")



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# print current working directory
!pwd

/content


In [8]:
#Configurations
docs_dir_path = "/content/doc_dir/doc_dir"
vector_db_path = "/content/vector_db"
collection_name = "documents_collection"

In [9]:
#define embedding function (download& load)
embed_model = HuggingFaceEmbedding()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
#directory loader
loader = SimpleDirectoryReader(input_dir='/content/doc_dir/doc_dir')


In [11]:
#load the documents
documents = loader.load_data()

In [12]:
print(len(documents))

31


In [13]:
documents[0]

Document(id_='0bebf7a7-9437-49fa-a6d9-c8c3abf41700', embedding=None, metadata={'page_label': '1', 'file_name': 'Attention.pdf', 'file_path': '/content/doc_dir/doc_dir/Attention.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2025-10-16', 'last_modified_date': '2025-10-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@goog

In [14]:
documents[1]

Document(id_='3cfa616e-672d-4bcd-a800-e28f90cc1098', embedding=None, metadata={'page_label': '2', 'file_name': 'Attention.pdf', 'file_path': '/content/doc_dir/doc_dir/Attention.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2025-10-16', 'last_modified_date': '2025-10-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='1 Introduction\nRecurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks\nin particular, have been firmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 35, 2, 

In [15]:
#create parser with chunking stratagey
parser = SimpleNodeParser.from_defaults(chunk_size=1024,chunk_overlap=50)


In [16]:
#convert documents to chunks(nodes)
nodes = parser.get_nodes_from_documents(documents)

In [17]:
#Define persistant DB loaction
db = chromadb.PersistentClient(path=vector_db_path)

In [18]:
#create or retrive the vector collection
chroma_collection = db.get_or_create_collection(name=collection_name)

In [19]:
#create vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [20]:
# create stroage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [21]:
# Create the vector store index

index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    vector_store=vector_store,
    embed_model=embed_model,
)

In [22]:
print("Vector store Index created")

Vector store Index created


LLAMAINDEX- RAG - Retrieval & Generation

In [23]:
#importing the dependencies

import os

from llama_index.core import VectorStoreIndex,StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core import Settings
import chromadb

In [24]:
#Set API key
os.environ["GROQ_API_KEY"] = "your_GROQ_API_KEY"

In [25]:
#set LLM and embeddings
Settings.llm = Groq(model="llama-3.1-8b-instant")

In [26]:
embed_model = HuggingFaceEmbedding()

In [27]:
vector_db_path = "/content/vector_db"
collection_name = "documents_collection"

In [28]:
#Load Chroma Collection
db = chromadb.PersistentClient(path=vector_db_path)
chroma_collection = db.get_collection(name=collection_name)

In [29]:
#connect to the vectore store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [30]:
#Load index from Chroma

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context = storage_context,
    embed_model=embed_model
)

In [31]:
#create the query engine

query_engine = index.as_query_engine(similarity_top_k=3)

In [32]:
#ask your question
response = query_engine.query("What are the key differences between BERT and Transformer")
print(response.response)

BERT and Transformer are two different models with distinct architectures. The key differences between them lie in their pre-training objectives and the way they process input sequences.

Transformer is a left-to-right model that uses a self-attention mechanism to process input sequences. It can only attend to previous tokens in the sequence, which limits its ability to capture context from both directions.

On the other hand, BERT is a bidirectional model that uses a "masked language model" (MLM) pre-training objective. It randomly masks some of the tokens in the input sequence and trains the model to predict the original vocabulary id of the masked token. This allows BERT to jointly condition on both left and right context in all layers, making it more effective for tasks that require capturing context from both directions.

In terms of architecture, BERT uses a bidirectional Transformer, whereas Transformer uses a left-to-right Transformer. This difference in architecture enables BE

In [33]:
response.metadata

{'fceaf168-5fa0-44fa-8593-7ff9cbd7af82': {'page_label': '8',
  'file_name': 'BERT.pdf',
  'file_path': '/content/doc_dir/doc_dir/BERT.pdf',
  'file_type': 'application/pdf',
  'file_size': 775166,
  'creation_date': '2025-10-16',
  'last_modified_date': '2025-10-16'},
 'fef02ed4-923d-4f53-8b96-eb13c84aeb57': {'page_label': '1',
  'file_name': 'BERT.pdf',
  'file_path': '/content/doc_dir/doc_dir/BERT.pdf',
  'file_type': 'application/pdf',
  'file_size': 775166,
  'creation_date': '2025-10-16',
  'last_modified_date': '2025-10-16'},
 '53821502-19e6-43b6-8b8e-f596787641cc': {'page_label': '2',
  'file_name': 'BERT.pdf',
  'file_path': '/content/doc_dir/doc_dir/BERT.pdf',
  'file_type': 'application/pdf',
  'file_size': 775166,
  'creation_date': '2025-10-16',
  'last_modified_date': '2025-10-16'}}

In [35]:
# Ask your question
response = query_engine.query("How is BERT pre-trained?")

print("Answer:")
print(response.response)

print("\nSources:")
if hasattr(response, "metadata") and response.metadata:
    file_names = {meta["file_name"] for meta in response.metadata.values()}
    for f in file_names:
        print("-", f)

Answer:
BERT is pre-trained using a "masked language model" (MLM) objective, inspired by the Cloze task. The MLM objective randomly masks some of the tokens from the input, and the model is trained to predict the original vocabulary id of the masked token based on its context.

Sources:
- BERT.pdf


In [36]:
# Ask your question
response = query_engine.query("What are query, key, and value in attention mechanism?")

print("Answer:")
print(response.response)

print("\nSources:")
if hasattr(response, "metadata") and response.metadata:
    file_names = {meta["file_name"] for meta in response.metadata.values()}
    for f in file_names:
        print("-", f)

Answer:
In the context of an attention mechanism, the query, key, and value are vectors that are used to compute the output. The query is a vector that is used to compute the weights for the attention function, the key is a vector that is used to compute the compatibility function with the query, and the value is a vector that is used to compute the output.

Sources:
- Attention.pdf
